# Revisión Solver de Markowitz





**Código revisado**

## Librerias necesarias

In [1]:
#Librerias necesarias
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  22571      0 --:--:-- --:--:-- --:--:-- 22571
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


In [2]:

import cupy as cp
import numpy as np
import pandas as pd
import fix_yahoo_finance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import time

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


A continuación creamos un arreglo con las acciones que utilizaremos.

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [0]:
def extraer_datos_yahoo(stocks):
  '''
  Funcion para extraer datos de los portafolios de yahoo finance de 2015-01-01 a 2020-04-30
  '''
  df_c = yf.download(stocks, start='2015-01-01', end='2020-04-30').Close
  base = df_c['AAPL'].dropna().to_frame()
  for i in range(0,50):
      base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
  base = base.drop(columns=['AAPL_caller'])
  base = base.rename(columns={"AAPL_other": "AAPL"})
  base = base.fillna(method='ffill')
  base = base.fillna(method='bfill')
  return base

In [5]:
datos = extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


Revisamos los filas de los datos

In [6]:
datos

,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,BABA,BHP.AX,BRK-A,CCI,COP,CVX,DLR,FB,GOOG,HD,HON,JNJ,JPM,KO,LIN,LMT,MA,MC.PA,MRK,MSFT,NSRGY,PEP,PFE,PG,PLD,RDSA.AS,RELIANCE.NS,RHHBY,RIO.L,RTX,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26600.0,24.704800,112.800003,5.77,79.500000,7507.0,109.330002,89.786400,99.669998,308.519989,103.599998,27.603399,223600.0,79.510002,68.919998,112.580002,66.410004,78.449997,523.373108,103.430000,95.556229,104.519997,62.490002,42.139999,129.949997,193.309998,85.680000,130.850006,57.189999,46.759998,72.650002,94.440002,31.330000,90.440002,43.430000,27.750000,442.774994,33.910000,2970.0,72.397736,33.869999,100.779999,118.610001,110.379997,66.254997,188.399994,105.919998,46.959999,85.900002,92.830002
2015-01-05,26660.0,24.951799,113.500000,5.80,79.500000,7507.0,106.250000,87.005997,98.230003,302.190002,101.000000,27.547300,220980.0,79.000000,65.639999,108.080002,67.690002,77.190002,512.463013,101.260002,93.735291,103.790001,60.549999,42.139999,126.519997,189.289993,83.269997,127.050003,58.040001,46.330002,70.959999,93.730003,31.160000,90.010002,43.400002,26.615000,437.924988,34.029999,2883.5,71.189430,33.549999,99.120003,114.599998,108.169998,64.792503,185.089996,104.099998,46.570000,85.650002,90.290001
2015-01-06,25900.0,24.605900,120.000000,5.71,77.000000,7300.0,106.260002,86.279999,97.970001,295.290009,103.320000,26.267099,220450.0,78.849998,62.930000,108.029999,67.480003,76.150002,500.585632,100.949997,93.516014,103.279999,58.980000,42.459999,124.900002,188.399994,83.089996,125.599998,60.320000,45.650002,70.610001,93.019997,31.420000,89.599998,43.549999,26.514999,418.049988,33.900002,2944.5,70.182503,33.599998,98.919998,112.230003,107.459999,64.375000,183.270004,103.080002,47.040001,86.309998,89.809998
2015-01-07,26140.0,24.507099,124.400002,5.75,78.250000,7407.0,107.750000,86.669601,99.000000,298.420013,102.129997,26.267099,223480.0,80.500000,63.349998,107.940002,68.019997,76.150002,499.727997,104.410004,94.192909,105.559998,59.070000,42.990002,126.300003,190.830002,84.220001,125.699997,61.610001,46.230000,70.750000,95.739998,31.850000,90.070000,44.209999,26.870001,427.149994,33.990002,2962.5,70.943993,33.169998,99.930000,112.849998,108.459999,65.237503,185.559998,104.309998,46.189999,88.599998,90.720001
2015-01-08,26280.0,23.864799,127.300003,5.72,79.250000,7554.0,111.889999,90.317703,99.919998,300.459991,105.029999,26.519400,226680.0,81.769997,64.930000,110.410004,68.910004,78.180000,501.303680,106.720001,95.908974,106.389999,60.389999,43.509998,128.380005,195.130005,85.529999,129.649994,62.849998,47.590000,71.459999,97.480003,32.500000,91.099998,44.220001,27.495001,421.024994,34.279999,3027.5,72.152298,33.500000,104.699997,117.080002,110.410004,66.112503,188.820007,106.150002,47.180000,90.470001,92.230003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-23,52100.0,26.200001,411.600006,5.06,70.300003,6570.0,275.029999,117.599998,248.600006,2399.449951,205.240005,29.750000,278750.0,160.550003,36.180000,86.800003,148.529999,185.130005,1276.310059,202.320007,134.679993,155.509995,89.389999,45.070000,181.300003,376.730011,255.860001,348.649994,80.879997,171.419998,106.000000,130.259995,36.689999,119.400002,86.750000,16.521999,1370.900024,44.180000,3783.0,63.290001,29.500000,285.329987,152.289993,99.449997,166.380005,256.420013,139.550003,57.590000,128.529999,43.450001
2020-04-24,52100.0,26.100000,406.399994,5.06,70.300003,6543.0,282.970001,115.300003,244.610001,2410.219971,204.360001,30.540001,279460.0,161.610001,36.090000,87.010002,150.029999,190.070007,1279.310059,212.179993,135.520004,154.860001,90.709999,45.430000,181.470001,381.769989,258.760010,340.850006,81.430000,174.550003,109.220001,134.360001,37.380001,118.779999,89.040001,15.978000,1417.000000,45.

## Funciones auxiliares


In [0]:
def calcular_rendimiento_vector(x):
  """
  Función para calcular el rendimiento esperado

  params:
      x     vector de precios
  
  return:
      r_est rendimiento esperado diario
  """

  # Definimos precios iniciales y finales como arreglo alojado en la gpu
  x_o = cp.asarray(x)
  x_f = x_o[1:]

  # Calculamos los rendimientos diarios
  r = cp.log(x_f/x_o[:-1])

  return r

In [0]:
def calcular_rendimiento(X):
  """
  Función para calcular el rendimiento esperado para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
             n el número de acciones
  
  return:
      r_est rvector de rendimientos esperados
  """
  m,n = X.shape
  r_est = cp.zeros(n)
  X = cp.asarray(X)

  for i in range(n):
    r_est[i] = calcular_rendimiento_vector(X[:,i]).mean()

  return 264*r_est

In [0]:
def calcular_varianza(X):

  """
  Función para calcular el la matriz de varianzas y covarianzas para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
               n el número de acciones
  
  return:
      S  matriz de varianzas y covarianzas
  """
  m,n=X.shape
  X = cp.asarray(X)

  X_m = cp.zeros((m-1,n))

  for i in range(n):
    X_m[:,i] = calcular_rendimiento_vector(X[:,i]) - calcular_rendimiento_vector(X[:,i]).mean()

  S = (cp.transpose(X_m)@X_m)/(m-2)

  return S

## Solución del modelo de Markowitz

In [0]:
def formar_vectores(mu, Sigma):
  '''
  Calcula las cantidades u = \Sigma^{-1}  \mu y v := \Sigma^{-1} \cdot 1 del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    u (cupy array, escalar): vector dado por \cdot Sigma^-1 \cdot mu (dimension n)
    v (cupy array, escalar): vector dado por Sigma^-1 \cdot 1 (dimension n)
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  # Obtiene vectores de interes
  u = cp.linalg.solve(Sigma, mu)
  u = u.transpose() # correcion de expresion de array
  v = cp.linalg.solve(Sigma, ones_vector)

  return u , v

In [0]:
def formar_abc(mu, Sigma):
  '''
  Calcula las cantidades A, B y C del diagrama de flujo del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  u, v = formar_vectores(mu, Sigma)

  # Obtiene escalares de interes
  A = mu.transpose()@u
  B = ones_vector.transpose()@v
  C = ones_vector.transpose()@u

  return A, B, C

In [0]:
def delta(A,B,C):
  '''
  Calcula las cantidad Delta = AB-C^2 del diagrama de flujo del problema de Markowitz

  Args:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu

  Return:
    Delta (cupy array, escalar): escalar dado \mu^t \cdot \Sigma^{-1} \cdot \mu
  '''
  Delta = A*B-C**2

  return Delta

In [0]:
def formar_omegas(r, mu, Sigma):
  '''
  Calcula las cantidades w_o y w_ del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    w_0 (cupy array, matriz): matriz dada por 
          w_0 = \frac{1}{\Delta} (B \Sigma^{-1} \hat{\mu}- C\Sigma^{-1} 1) 
    w_1 (cupy array, vector): vector dado por 
         w_1 = \frac{1}{\Delta} (C \Sigma^{-1} \hat{\mu}- A\Sigma^{-1} 1)
  '''
  # Obtenemos u = Sigma^{-1} \hat{\mu}, v = \Sigma^{-1} 1
  u, v = formar_vectores(mu, Sigma)
  # Escalares relevantes
  A, B, C = formar_abc(mu, Sigma)
  Delta = delta(A,B,C)
  # Formamos w_0 y w_1
  w_0 = (1/Delta)*(r*B-C)
  w_1 = (1/Delta)*(A-C*r)

  return w_0, w_1


In [0]:
def markowitz(r, mu, Sigma):
  '''
  Calcula las cantidades w_o y w_ del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    w_0 (cupy array, matriz): matriz dada por 
          w_0 = \frac{1}{\Delta} (B \Sigma^{-1} \hat{\mu}- C\Sigma^{-1} 1) 
    w_1 (cupy array, vector): vector dado por 
         w_1 = \frac{1}{\Delta} (C \Sigma^{-1} \hat{\mu}- A\Sigma^{-1} 1)
  '''
  # Obtenemos u = Sigma^{-1} \hat{\mu}, v = \Sigma^{-1} 1
  u, v = formar_vectores(mu, Sigma)

  # Formamos w_0 y w_1
  w_0, w_1 = formar_omegas(r, mu, Sigma)

  return w_0*u+w_1*v


## Revisón

**1. Documentación**

La Documentación expresa de manera clara, consica y breve lo que hace el código. De igual forma se explica de manera clara y concisa los argumentos de entrada y salida. La documentación es completa.

**2. Cumplimiento de objetivos del código**

La función cumple con el objetivo devolviendo una matriz y un vector de pesos w.

**3. Test**

Objetivo: verificar el desempeño del código con diferentes números de activos y rendimientos, y verificar que los pesos sumen 1.


In [0]:
datos_1 = cp.random.uniform(1,1000,10**3).reshape(10**2,10)
sigma_1 = calcular_varianza(datos_1)
mu_1 = calcular_rendimiento(datos_1)
r_11 = cp.mean(mu_1)**2
r_12 = max(mu_1)

In [0]:
w_11 = markowitz(r_11,mu_1,sigma_1)
w_12 = markowitz(r_12,mu_1,sigma_1)

Verificamos que la suma de las $\sum w's=1$ y el error absoluto

In [18]:
sum(w_11)

array(1.)

In [19]:
abs((sum(w_11)-1)/sum(w_11))

array(1.11022302e-16)

In [20]:
sum(w_12)

array(1.)

In [21]:
abs((sum(w_12)-1)/sum(w_12))

array(4.4408921e-16)

Verificamos el error absoluto de $w^{t}\mu = r$


In [22]:
error_11 = abs(r_11 - w_11@mu_1)/r_11
print(error_11)

6.155967883235464e-16


In [23]:
error_12 = abs(r_12 - w_12@mu_1)/r_12
print(error_12)

1.2309415527669263e-16


In [24]:
w_11@mu_1

array(1.44279248)

In [25]:
r_11

array(1.44279248)

In [26]:
w_12@mu_1

array(3.60771971)

In [27]:
r_12

array(3.60771971)

Ahora probamos con una matriz de $10^4 \times 10^2$

In [0]:
datos_2 = cp.random.uniform(1,1000,10**6).reshape(10**4,10**2)
sigma_2 = calcular_varianza(datos_2)
mu_2 = calcular_rendimiento(datos_2)
r_21 = cp.mean(mu_2)**2
r_22 = max(mu_2)

In [0]:
w_21 = markowitz(r_21,mu_2,sigma_2)
w_22 = markowitz(r_22,mu_2,sigma_2)

Verificamos que la suma de las $\sum w's=1$ y el error absoluto

In [30]:
sum(w_21)

array(1.)

In [31]:
abs((sum(w_21)-1)/sum(w_21))

array(2.22044605e-16)

In [32]:
sum(w_22)

array(1.)

In [33]:
abs((sum(w_22)-1)/sum(w_22))

array(0.)

Verificamos el error absoluto de $w^{t}\mu = r$

In [34]:
w_21@mu_2

array(4.18727825e-05)

In [35]:
r_21

array(4.18727825e-05)

In [36]:
error_21 = abs(r_21 - w_21@mu_2)/r_21
print(error_21)

1.304348104457549e-13


In [37]:
error_22 = abs(r_22 - w_22@mu_2)/r_22
print(error_22)

3.770752517849849e-16


Ahora probamos con una matriz de $10^5 \times 10^3$



In [0]:
datos_3 = cp.random.uniform(1,1000,10**8).reshape(10**5,10**3)
sigma_3 = calcular_varianza(datos_3)
mu_3 = calcular_rendimiento(datos_3)
r_31 = cp.mean(mu_3)**2
r_32 = max(mu_3)

In [0]:
w_31 = markowitz(r_31,mu_3,sigma_3)
w_32 = markowitz(r_32,mu_3,sigma_3)

Verificamos que la suma de las $\sum w's=1$ y el error absoluto

In [40]:
sum(w_31)

array(1.)

In [41]:
abs(sum(w_31)-1)/sum(w_31)

array(6.66133815e-16)

In [42]:
sum(w_32)

array(1.)

Verificamos el error absoluto de $w^{t}\mu = r$

In [43]:
error_31 = abs(r_31 - w_31@mu_3)/r_31
print(error_31)

5.972565684253623e-12


In [44]:
error_32 = abs(r_32 - w_32@mu_3)/r_32
print(error_32)

1.2765462539881994e-16


Ahora probamos con una matriz de $10^4 \times 10^4$

In [0]:
datos_4 = cp.random.uniform(100,1000,10**8).reshape(10**4,10**4)
sigma_4 = calcular_varianza(datos)
mu_4 = calcular_rendimiento(datos)
r_41 = cp.mean(mu_4)**2
r_42 = max(mu_4)

In [0]:
w_41 = markowitz(r_41,mu_4,sigma_4)
w_42 = markowitz(r_42,mu_4,sigma_4)

Verificamos que  ∑w′s=1 y el error absoluto

In [47]:
sum(w_41)

array(1.)

In [48]:
abs(sum(w_41)-1)/sum(w_41)

array(4.4408921e-16)

In [49]:
sum(w_42)

array(1.)

In [50]:
abs(sum(w_42)-1)/sum(w_42)

array(5.55111512e-16)

Verificamos el error absoluto de $w^{t}\mu = r$

In [51]:
error_41 = abs(r_41 - w_41@mu_4)/r_41
print(error_41)

6.689301079114241e-14


In [52]:
error_42 = abs(r_42 - w_42@mu_4)/r_42
print(error_42)

1.2421353758545672e-15


In [53]:
r_41

array(0.00793544)

In [54]:
w_41@mu_4

array(0.00793544)

In [56]:
r_42

array(0.40221088)

In [55]:
w_42@mu_4

array(0.40221088)

Ahora probamos con una matriz de $10^5 \times 10^4$

In [57]:
datos_5 = cp.random.uniform(1,1000,10**9).reshape(10**5,10**4)
sigma_5 = calcular_varianza(datos_5)
mu_5 = calcular_rendimiento(datos_5)
r_51 = cp.mean(mu_5)**2
r_52 = max(mu_5)

OutOfMemoryError: ignored

EL sistema no soporta generar matrices con 10**9 elementos.

Probamos con las acciones



In [0]:
sigma = calcular_varianza(datos)
mu = calcular_rendimiento(datos)
r1 = cp.mean(mu)**2
r2 = max(mu)

In [0]:
w_01 = markowitz(r1,mu,sigma)
w_02 = markowitz(r2,mu,sigma)

Verificamos que  ∑w′s=1 y el error absoluto

In [60]:
sum(w_01)

array(1.)

In [61]:
abs(sum(w_01)-1)/sum(w_01)

array(4.4408921e-16)

In [62]:
sum(w_02)

array(1.)

In [63]:
abs(sum(w_02)-1)/sum(w_02)

array(5.55111512e-16)

Verificamos el error absoluto de $w^{t}\mu = r$

In [65]:
error_01 = abs(r1 - w_01@mu)/r1
print(error_01)

6.689301079114241e-14


In [67]:
error_02 = abs(r2 - w_02@mu)/r2
print(error_02)

1.2421353758545672e-15


# Hallazgos

El código funciona de manera correcta para distintos tamaños de matrices y para el protafolio de acciones.

Las funciones generan unos valores con una exactitud de hasta 16 dígitos correctos. 


**Nota**: Los tests fueron realizados en google colaboratory con GPU como entorno de ejecución